IDEA: QUEDARME CON LOS USUARIOS QUE MAYOR CANTIDAD DE ANIMES VIERON Y ENTRENAR UN MODELO SOLO CON ESOS USUARIOS

In [1]:
#Importamos las librerias que utilizaremos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
rating_complete = pd.read_csv(r"C:\Users\ferbi\OneDrive\Desktop\Fer\DATASCIENCE\Cuarto TP Sistemas de recomendacion\archive (10)\rating_complete.csv")

In [3]:
rating_complete.head()

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


In [4]:
rating_complete.shape

(57633278, 3)

In [6]:
rating_complete.user_id.nunique() #Tenemos 310.059 usuarios unicos 

310059

In [7]:
# Calculamos la cantidad de animes vistos por cada usuario
anime_count_by_user = rating_complete.groupby('user_id')['anime_id'].count()
anime_count_by_user

user_id
0          35
1         103
2          51
3         315
4         118
         ... 
353400     67
353401     62
353402     19
353403     67
353404     30
Name: anime_id, Length: 310059, dtype: int64

In [8]:
# Ordenamos los datos en orden descendente
anime_count_by_user_sorted = anime_count_by_user.sort_values(ascending=False)

# Convertimos la serie ordenada en un nuevo DataFrame
df_anime_count_sorted = pd.DataFrame({'user_id': anime_count_by_user_sorted.index, 'amount_animes_watched': anime_count_by_user_sorted.values})

# Imprimimos el DataFrame ordenado
print("Usuarios ordenados por cantidad de animes vistos:")
print(df_anime_count_sorted)

Usuarios ordenados por cantidad de animes vistos:
        user_id  amount_animes_watched
0        189037                  15455
1        162615                  14864
2         68042                  13462
3        283786                  12778
4        259790                   9996
...         ...                    ...
310054   201316                      1
310055   201300                      1
310056    97555                      1
310057   201293                      1
310058   137466                      1

[310059 rows x 2 columns]


In [10]:
df_anime_count_sorted.shape #Tenemos 310.059 usuarios diferentes 

(310059, 2)

Supongamos que me quedo con los 1000 usuarios que mayor cantidad de animes vieron, veamos el tamaño del set de datos original, filtrado segun estos usuarios.

In [11]:
df_1000 = df_anime_count_sorted.head(1000)
df_1000

,user_id,amount_animes_watched
0,189037,15455
1,162615,14864
2,68042,13462
3,283786,12778
4,259790,9996
...,...,...
995,325019,1673
996,12044,1671
997,332795,1671
998,222269,1671


In [12]:
rating_complete.head()

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


Unimos ambos datasets: 

In [13]:
# Realizamos la unión utilizando 'user_id' como clave y 'inner' como método
merged_df = pd.merge(rating_complete, df_1000, on='user_id', how='inner')

# Imprimimos el DataFrame resultante
print("Usuarios con 'user_id' en común:")
print(merged_df)

Usuarios con 'user_id' en común:
         user_id  anime_id  rating  amount_animes_watched
0            781     10122       6                   3062
1            781       851      10                   3062
2            781     15005       8                   3062
3            781     34955       5                   3062
4            781     20603       7                   3062
...          ...       ...     ...                    ...
2475590   352930       671       7                   2300
2475591   352930      1017       8                   2300
2475592   352930     10465       6                   2300
2475593   352930     10845       6                   2300
2475594   352930     14045       7                   2300

[2475595 rows x 4 columns]


In [14]:
merged_df.shape

(2475595, 4)

In [15]:
merged_df.head()

,user_id,anime_id,rating,amount_animes_watched
0,781,10122,6,3062
1,781,851,10,3062
2,781,15005,8,3062
3,781,34955,5,3062
4,781,20603,7,3062


Dividimos el dataset en conjuntos de train y test: 

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
df_train, df_test = train_test_split(merged_df, test_size=0.2, random_state=42)

In [18]:
df_train.head()

,user_id,anime_id,rating,amount_animes_watched
1410552,205655,19023,5,1836
1203513,174711,6772,6,2864
1660878,245012,12191,4,2001
1193075,173405,28957,10,1709
1169418,169563,5152,7,3141


In [19]:
df_train.shape #Tenemos un conjunto de entrenamiento con 1.980.476 entradas

(1980476, 4)

In [20]:
df_train.user_id.nunique() #En el set de train tenemos 1000 usuarios diferentes

1000

In [21]:
df_train.anime_id.nunique() #En el set de train tenemos 16.795 animes diferentes

16795

In [22]:
df_test.head()

,user_id,anime_id,rating,amount_animes_watched
269048,36253,7674,9,1674
915920,128855,34107,7,1834
1971616,286479,25731,7,1703
1510376,221161,31646,7,3542
84176,11697,40540,8,2777


In [23]:
df_test.shape #Tenemos un conjunto de testeo con 495.119 entradas

(495119, 4)

In [24]:
df_test.user_id.nunique() #En el set de test tenemos 1000 usuarios diferentes 

1000

Dado que en todo el set de datos tenemos 1000 usuarios diferentes, entendemos que en este caso no habra usuarios COLD START, ya que todos los usuarios que se encuentran en train, se encuentran tambien en test

In [25]:
#Comprobemos esto:
df_test[~df_test.user_id.isin(df_train.user_id.unique())].user_id.nunique()

0

Efectivamente, no tenemos usuarios que aparecen en el set de test pero no en el de train. Sino que todos los usuarios aparecen en ambos conjuntos.  


In [26]:
df_train.columns

Index(['user_id', 'anime_id', 'rating', 'amount_animes_watched'], dtype='object')

In [27]:
#Nos quedamos con las columnas que nos interesan, tanto en train como en test: 
df_train = df_train.drop('amount_animes_watched', axis=1)
df_test = df_test.drop('amount_animes_watched', axis=1)

In [28]:
df_train.head()

,user_id,anime_id,rating
1410552,205655,19023,5
1203513,174711,6772,6
1660878,245012,12191,4
1193075,173405,28957,10
1169418,169563,5152,7


In [29]:
df_test.head()

,user_id,anime_id,rating
269048,36253,7674,9
915920,128855,34107,7
1971616,286479,25731,7
1510376,221161,31646,7
84176,11697,40540,8


Ahora, armamos la matriz de interacciones. Sabemos que tenemos 1000 usuarios diferentes y 16.795 animes diferentesen el conjunto de train. 

In [30]:
matrix_columns = ['user_id', 'anime_id', 'rating']

In [31]:
interactions = df_train[matrix_columns]
interactions.head()

,user_id,anime_id,rating
1410552,205655,19023,5
1203513,174711,6772,6
1660878,245012,12191,4
1193075,173405,28957,10
1169418,169563,5152,7


In [32]:
interactions.shape

(1980476, 3)

In [33]:
#Reseteamos los indices
interactions = interactions.reset_index(drop=True)

In [34]:
interactions.head()

,user_id,anime_id,rating
0,205655,19023,5
1,174711,6772,6
2,245012,12191,4
3,173405,28957,10
4,169563,5152,7


In [35]:
#Verificamos si existen filas duplicadas
duplicate_rows = interactions[interactions.duplicated(subset=['user_id', 'anime_id'], keep=False)]
print(duplicate_rows)

Empty DataFrame
Columns: [user_id, anime_id, rating]
Index: []


No existen filas duplicadas

In [ ]:
interactions_matrix = interactions.pivot_table(index='user_id', columns='anime_id', values='rating')

In [ ]:
interactions_matrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,47307,47398,47402,47403,47410,47614,47616,47618,48177,48456
user_id,,,,,,,,,,,,,,,,,,,,,
781,NaN,NaN,10.0,10.0,NaN,NaN,7.0,NaN,8.0,10.0,...,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
890,NaN,9.0,9.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
943,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN
1177,8.0,8.0,7.0,NaN,NaN,NaN,7.0,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1397,8.0,8.0,8.0,7.0,NaN,NaN,8.0,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Completamos los NaN con 0s
interactions_matrix = interactions_matrix.fillna(0)
interactions_matrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,47307,47398,47402,47403,47410,47614,47616,47618,48177,48456
user_id,,,,,,,,,,,,,,,,,,,,,
781,0.0,0.0,10.0,10.0,0.0,0.0,7.0,0.0,8.0,10.0,...,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
890,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
943,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
1177,8.0,8.0,7.0,0.0,0.0,0.0,7.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1397,8.0,8.0,8.0,7.0,0.0,0.0,8.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
interactions_matrix.shape #Tiene 1000 filas (usuarios) y 16.795 columnas (animes)

(1000, 16795)

Vamos a transformar esta matriz sparsa a una matriz csr matrix (compressed sparse row)

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
interactions_matrix_csr = csr_matrix(interactions_matrix.values)

In [ ]:
interactions_matrix_csr

<1000x16795 sparse matrix of type '<class 'numpy.float64'>'
	with 1980476 stored elements in Compressed Sparse Row format>

La siguiente celda generara un diccionario que nos servira mas adelante como indice. Tiene como keys el id de cada usuario y como valor el indice (numero de fila de la matriz)

In [ ]:
interactions_matrix.index

Index([   781,    890,    943,   1177,   1397,   1469,   1946,   3578,   4773,
         5045,
       ...
       350166, 350215, 350286, 351119, 351361, 351696, 351733, 351801, 352922,
       352930],
      dtype='int64', name='user_id', length=1000)

In [ ]:
user_ids = list(interactions_matrix.index)
user_dict = {}
counter = 0 
for i in user_ids:
    user_dict[i] = counter
    counter += 1

In [ ]:
user_dict

{781: 0,
 890: 1,
 943: 2,
 1177: 3,
 1397: 4,
 1469: 5,
 1946: 6,
 3578: 7,
 4773: 8,
 5045: 9,
 5648: 10,
 6149: 11,
 6548: 12,
 6783: 13,
 7179: 14,
 7588: 15,
 8074: 16,
 8326: 17,
 9528: 18,
 9615: 19,
 9740: 20,
 9747: 21,
 10255: 22,
 10268: 23,
 10359: 24,
 10367: 25,
 10409: 26,
 10851: 27,
 10869: 28,
 11249: 29,
 11648: 30,
 11697: 31,
 11716: 32,
 12044: 33,
 12688: 34,
 12888: 35,
 13184: 36,
 13259: 37,
 13307: 38,
 14347: 39,
 14455: 40,
 14874: 41,
 15083: 42,
 15609: 43,
 16057: 44,
 16282: 45,
 16323: 46,
 16766: 47,
 17273: 48,
 17363: 49,
 18355: 50,
 19014: 51,
 19064: 52,
 19413: 53,
 19486: 54,
 19497: 55,
 20314: 56,
 20457: 57,
 20546: 58,
 20782: 59,
 20807: 60,
 21411: 61,
 21615: 62,
 21878: 63,
 22018: 64,
 23184: 65,
 23878: 66,
 23930: 67,
 24099: 68,
 24350: 69,
 24725: 70,
 24833: 71,
 24875: 72,
 24983: 73,
 24990: 74,
 25383: 75,
 25411: 76,
 25898: 77,
 25902: 78,
 26045: 79,
 26650: 80,
 26654: 81,
 26661: 82,
 26807: 83,
 27297: 84,
 27413: 85,
 27

Modelado: ya tenemos la matriz lista para entrenar el modelo. Utilizaremos la libreria LFM (light fm)

In [ ]:
pip install lightfm 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from lightfm import LightFM

c:\Users\ferbi\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


Utilizaremos GridSearchCV para obtener los mejores parametros: 
- no_components: [3, 10, 50, 100, 200] (caracteristicas latentes)
- learning_rate:  [0.01, 0.03, 0.1, 0.2] (controla cuanto se ajustan los parametros del modelo en cada paso del entrenamiento)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.03, 0.1, 0.2],
    'no_components': [3, 10, 50, 100, 200]
}

In [ ]:
model = LightFM()# Instanciamos el modelo

In [ ]:
#grid_search = GridSearchCV(model, param_grid, scoring='accuracy', cv=5)

In [ ]:
#grid_search.fit(interactions_matrix_csr, epochs=10, verbose=True)

Epoch: 100%|██████████| 10/10 [00:12<00:00,  1.24s/it]
c:\Users\ferbi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\ferbi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 811, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
Epoch: 100%|██████████| 10/10 [1:38:47<00:00, 592.78s/it]  
c:\Users\ferbi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]
  warnings.warn(
Epoch: 100%|████████

GridSearchCV(cv=5,
             estimator=<lightfm.lightfm.LightFM object at 0x00000181A789CE10>,
             param_grid={'learning_rate': [0.01, 0.03, 0.1, 0.2],
                         'no_components': [3, 10, 50, 100, 200]},
             scoring='accuracy')

In [ ]:
#best_params = grid_search.best_params_
#best_params

{'learning_rate': 0.01, 'no_components': 3}

Los mejores parametros encontrados fueron:
no_components = 3
learning_rate = 0.01

Conociendo los mejores parametros, obtenidos a partir de GridSearchCV, entrenamos el modelo con estos parametros:

In [ ]:
model = LightFM(no_components=3, random_state=100, learning_rate=0.01)

Entrenamos este modelo durante 10 epocas:

In [ ]:
%%time 
#Devuelve el tiempo que le lleva a mi maquina el entrenamiento
model = model.fit(interactions_matrix_csr, epochs = 10)

CPU times: total: 5.22 s
Wall time: 15.3 s


Obtenemos las recomendaciones para cada usuario en el set de test, en este caso, al no tener usuarios cold start, tampoco haremos recomendaciones del contenido mas popular:

In [ ]:
from tqdm import tqdm

# Definimos dict donde vamos a ir almacenando las recomendaciones
recomms_dict = {
    'user_id': [],
    'recomms': [],
    'watched_content': []  # Nueva columna para contener el contenido que el usuario ya vio
}

# Obtenemos cantidad de usuarios y cantidad de items
n_users, n_items = interactions_matrix.shape
item_ids = np.arange(n_items)

# Por cada usuario del dataset de test, generamos recomendaciones
for user in tqdm(df_test.user_id.unique()):
    # Validar si el usuario se encuentra en la matriz de interacciones (interactions_matrix.index)
    if user in list(interactions_matrix.index):
        # Si el usuario está en train, no es cold start. Usamos el modelo para recomendar
        user_x = user_dict[user]  # Buscamos el índice del usuario en la matriz (transformamos id a índice)

        # Generar las predicciones para el usuario x
        preds = model.predict(user_ids=user_x, item_ids=item_ids)

        # Basándose en el ejemplo anterior, ordenar las predicciones de menor a mayor y quedarse con 50.
        scores = pd.Series(preds)
        scores.index = interactions_matrix.columns
        scores = list(pd.Series(scores.sort_values(ascending=False).index))[:50]

        # Obtener listado de contenidos vistos anteriormente por el usuario (en el set de train)
        watched_contents = df_train[df_train.user_id == user].anime_id.unique()

        # Filtrar contenidos ya vistos y quedarse con los primeros 10
        recomms = [x for x in scores if x not in watched_contents][:10]

        # Guardamos las recomendaciones en el diccionario
        recomms_dict['user_id'].append(user)
        recomms_dict['recomms'].append(scores)
        recomms_dict['watched_content'].append(watched_contents)

# Convertir el diccionario a un DataFrame
recomms_df = pd.DataFrame(recomms_dict)


100%|██████████| 1000/1000 [00:06<00:00, 161.88it/s]


In [ ]:
recomms_df = pd.DataFrame(recomms_dict)
recomms_df

,user_id,recomms,watched_content
0,36253,"[6547, 30276, 22319, 14813, 4224, 18153, 9756,...","[6324, 18195, 33502, 257, 2969, 18095, 9014, 3..."
1,128855,"[6547, 22319, 30276, 14813, 4224, 18153, 20507...","[3342, 32281, 2994, 21881, 16916, 1679, 6164, ..."
2,286479,"[6547, 22319, 30276, 14813, 4224, 18153, 20507...","[45, 24531, 387, 10491, 31798, 25859, 23985, 4..."
3,221161,"[6547, 22319, 30276, 14813, 18153, 20507, 4224...","[17251, 29501, 32238, 9332, 33, 21647, 31056, ..."
4,11697,"[6547, 22319, 30276, 14813, 4224, 18153, 28121...","[13855, 31715, 6984, 6736, 11021, 42990, 9253,..."
...,...,...,...
995,6149,"[6547, 30276, 14813, 4224, 22319, 9756, 18153,...","[9289, 33741, 38080, 18247, 21067, 13333, 9936..."
996,35941,"[6547, 30276, 14813, 4224, 22319, 18153, 9756,...","[3058, 3382, 169, 14353, 5040, 963, 238, 31452..."
997,217920,"[6547, 30276, 22319, 14813, 4224, 18153, 28121...","[101, 2899, 10681, 32698, 13839, 39988, 7222, ..."
998,135124,"[6547, 30276, 14813, 22319, 4224, 18153, 9756,...","[15633, 17873, 2889, 33219, 263, 37865, 15125,..."


EVALUACION DE METRICAS  
Compararemos nuestras recomendaciones contra las que los usuarios realemnte vieron (set de test)

Primero ordenamos los animes segun los ratings (de mayor a menor) que dieron los usuarios en el set de testeo. Luego, agrupamos y arnamamos un listado de animes para cada usuario. Este sera el listado "ideal" contra el que vams a comparar nuestras recomendaciones.

In [ ]:
ideal_recomms = df_test\
                  .sort_values(by=["user_id", "rating", "anime_id"], ascending=False)\
                  .groupby(["user_id"], as_index=False)\
                  .agg({"anime_id": lambda x: list(x.unique())})

Generamos el dataset ideal

In [ ]:
#Tenemos un conjunto de datos que muestra las recomendaciones
#ideales para cada uno de los usuarios del conjunto de testeo
ideal_recomms.head()

,user_id,anime_id
0,781,"[36553, 35843, 34096, 31043, 21333, 16738, 153..."
1,890,"[7025, 4240, 4181, 3220, 2981, 2257, 1088, 885..."
2,943,"[42984, 39940, 39324, 38889, 38680, 36480, 358..."
3,1177,"[10087, 5114, 3784, 3702, 2904, 2449, 1698, 14..."
4,1397,"[21557, 16528, 9000, 530, 270, 30, 33489, 2880..."


Vamos a medir MAP: 
Primero unimos en un mismo dataframe las recomndaciones con el set ideal 

In [ ]:
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="user_id", right_on="user_id")[["user_id", "anime_id", "recomms"]]
df_map.columns = ["user_id", "ideal", "recomms"]
df_map.head()

,user_id,ideal,recomms
0,781,"[36553, 35843, 34096, 31043, 21333, 16738, 153...","[6547, 22319, 30276, 20507, 14813, 18153, 4224..."
1,890,"[7025, 4240, 4181, 3220, 2981, 2257, 1088, 885...","[6547, 30276, 14813, 22319, 4224, 18153, 9756,..."
2,943,"[42984, 39940, 39324, 38889, 38680, 36480, 358...","[6547, 30276, 4224, 14813, 22319, 9756, 18153,..."
3,1177,"[10087, 5114, 3784, 3702, 2904, 2449, 1698, 14...","[6547, 30276, 14813, 22319, 4224, 18153, 9756,..."
4,1397,"[21557, 16528, 9000, 530, 270, 30, 33489, 2880...","[6547, 22319, 30276, 14813, 20507, 18153, 4224..."


In [ ]:
df_map.tail()

,user_id,ideal,recomms
995,351696,"[31149, 21939, 2251, 477, 202, 33818, 32983, 3...","[22319, 6547, 30276, 14813, 4224, 18153, 13601..."
996,351733,"[34591, 34076, 28957, 18679, 17909, 16918, 136...","[6547, 30276, 22319, 14813, 4224, 18153, 20507..."
997,351801,"[22535, 7193, 5040, 3014, 1535, 467, 338, 3392...","[6547, 30276, 22319, 14813, 20507, 4224, 18153..."
998,352922,"[16662, 283, 29831, 4081, 3297, 256, 15731, 46...","[6547, 30276, 4224, 14813, 22319, 9756, 5114, ..."
999,352930,"[13759, 12113, 9989, 9941, 6547, 6045, 5760, 5...","[6547, 30276, 14813, 4224, 22319, 18153, 9756,..."


In [ ]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["ideal", "recomms"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1 
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [ ]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.01544


Vemos que la metrica MAP si bien mejoró con respecto al sistema que utilizaba el 5% de datos seleccionado al azar, aún sigue sindo malo

Probamos entrenar el modelo durante 30 epocas: 

In [ ]:
%%time 
#Devuelve el tiempo que le lleva a mi maquina el entrenamiento
model = model.fit(interactions_matrix_csr, epochs = 30)

CPU times: total: 24.3 s
Wall time: 45.3 s


In [ ]:
from tqdm import tqdm

# Definimos dict donde vamos a ir almacenando las recomendaciones
recomms_dict = {
    'user_id': [],
    'recomms': [],
    'watched_content': []  # Nueva columna para contener el contenido que el usuario ya vio
}

# Obtenemos cantidad de usuarios y cantidad de items
n_users, n_items = interactions_matrix.shape
item_ids = np.arange(n_items)

# Por cada usuario del dataset de test, generamos recomendaciones
for user in tqdm(df_test.user_id.unique()):
    # Validar si el usuario se encuentra en la matriz de interacciones (interactions_matrix.index)
    if user in list(interactions_matrix.index):
        # Si el usuario está en train, no es cold start. Usamos el modelo para recomendar
        user_x = user_dict[user]  # Buscamos el índice del usuario en la matriz (transformamos id a índice)

        # Generar las predicciones para el usuario x
        preds = model.predict(user_ids=user_x, item_ids=item_ids)

        # Basándose en el ejemplo anterior, ordenar las predicciones de menor a mayor y quedarse con 50.
        scores = pd.Series(preds)
        scores.index = interactions_matrix.columns
        scores = list(pd.Series(scores.sort_values(ascending=False).index))[:50]

        # Obtener listado de contenidos vistos anteriormente por el usuario (en el set de train)
        watched_contents = df_train[df_train.user_id == user].anime_id.unique()

        # Filtrar contenidos ya vistos y quedarse con los primeros 10
        recomms = [x for x in scores if x not in watched_contents][:10]

        # Guardamos las recomendaciones en el diccionario
        recomms_dict['user_id'].append(user)
        recomms_dict['recomms'].append(scores)
        recomms_dict['watched_content'].append(watched_contents)

# Convertir el diccionario a un DataFrame
recomms_df = pd.DataFrame(recomms_dict)

100%|██████████| 1000/1000 [00:06<00:00, 158.21it/s]


In [ ]:
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="user_id", right_on="user_id")[["user_id", "anime_id", "recomms"]]
df_map.columns = ["user_id", "ideal", "recomms"]
df_map.head()

,user_id,ideal,recomms
0,781,"[36553, 35843, 34096, 31043, 21333, 16738, 153...","[4224, 6547, 16498, 10087, 9253, 10620, 23283,..."
1,890,"[7025, 4240, 4181, 3220, 2981, 2257, 1088, 885...","[4224, 6547, 16498, 10087, 9253, 8074, 10620, ..."
2,943,"[42984, 39940, 39324, 38889, 38680, 36480, 358...","[6547, 4224, 9253, 10087, 16498, 10620, 23283,..."
3,1177,"[10087, 5114, 3784, 3702, 2904, 2449, 1698, 14...","[6547, 4224, 16498, 10087, 9253, 8074, 10620, ..."
4,1397,"[21557, 16528, 9000, 530, 270, 30, 33489, 2880...","[4224, 6547, 16498, 10087, 9253, 23283, 10620,..."


In [ ]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["ideal", "recomms"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1 
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [ ]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.01377


Probamos entrenar el modelo durante 50 epocas: 

In [ ]:
%%time 
#Devuelve el tiempo que le lleva a mi maquina el entrenamiento
model = model.fit(interactions_matrix_csr, epochs = 50)

CPU times: total: 30.2 s
Wall time: 1min 16s


In [ ]:
from tqdm import tqdm

# Definimos dict donde vamos a ir almacenando las recomendaciones
recomms_dict = {
    'user_id': [],
    'recomms': [],
    'watched_content': []  # Nueva columna para contener el contenido que el usuario ya vio
}

# Obtenemos cantidad de usuarios y cantidad de items
n_users, n_items = interactions_matrix.shape
item_ids = np.arange(n_items)

# Por cada usuario del dataset de test, generamos recomendaciones
for user in tqdm(df_test.user_id.unique()):
    # Validar si el usuario se encuentra en la matriz de interacciones (interactions_matrix.index)
    if user in list(interactions_matrix.index):
        # Si el usuario está en train, no es cold start. Usamos el modelo para recomendar
        user_x = user_dict[user]  # Buscamos el índice del usuario en la matriz (transformamos id a índice)

        # Generar las predicciones para el usuario x
        preds = model.predict(user_ids=user_x, item_ids=item_ids)

        # Basándose en el ejemplo anterior, ordenar las predicciones de menor a mayor y quedarse con 50.
        scores = pd.Series(preds)
        scores.index = interactions_matrix.columns
        scores = list(pd.Series(scores.sort_values(ascending=False).index))[:50]

        # Obtener listado de contenidos vistos anteriormente por el usuario (en el set de train)
        watched_contents = df_train[df_train.user_id == user].anime_id.unique()

        # Filtrar contenidos ya vistos y quedarse con los primeros 10
        recomms = [x for x in scores if x not in watched_contents][:10]

        # Guardamos las recomendaciones en el diccionario
        recomms_dict['user_id'].append(user)
        recomms_dict['recomms'].append(scores)
        recomms_dict['watched_content'].append(watched_contents)

# Convertir el diccionario a un DataFrame
recomms_df = pd.DataFrame(recomms_dict)

100%|██████████| 1000/1000 [00:06<00:00, 157.47it/s]


In [ ]:
recomms_df = pd.DataFrame(recomms_dict)
recomms_df

,user_id,recomms,watched_content
0,36253,"[9253, 16498, 31964, 23283, 1575, 19815, 15809...","[6324, 18195, 33502, 257, 2969, 18095, 9014, 3..."
1,128855,"[9253, 16498, 30276, 31964, 23283, 15809, 1575...","[3342, 32281, 2994, 21881, 16916, 1679, 6164, ..."
2,286479,"[9253, 16498, 30276, 31964, 23283, 1575, 19815...","[45, 24531, 387, 10491, 31798, 25859, 23985, 4..."
3,221161,"[9253, 30276, 16498, 31964, 23283, 15809, 1981...","[17251, 29501, 32238, 9332, 33, 21647, 31056, ..."
4,11697,"[9253, 30276, 16498, 31964, 23283, 19815, 1575...","[13855, 31715, 6984, 6736, 11021, 42990, 9253,..."
...,...,...,...
995,6149,"[9253, 30276, 16498, 31964, 23283, 19815, 2904...","[9289, 33741, 38080, 18247, 21067, 13333, 9936..."
996,35941,"[9253, 16498, 31964, 30276, 23283, 19815, 1575...","[3058, 3382, 169, 14353, 5040, 963, 238, 31452..."
997,217920,"[9253, 30276, 16498, 31964, 23283, 19815, 1575...","[101, 2899, 10681, 32698, 13839, 39988, 7222, ..."
998,135124,"[9253, 16498, 30276, 31964, 23283, 15809, 1575...","[15633, 17873, 2889, 33219, 263, 37865, 15125,..."


In [ ]:
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="user_id", right_on="user_id")[["user_id", "anime_id", "recomms"]]
df_map.columns = ["user_id", "ideal", "recomms"]
df_map.head()

,user_id,ideal,recomms
0,781,"[36553, 35843, 34096, 31043, 21333, 16738, 153...","[9253, 31964, 16498, 23283, 30276, 1575, 19815..."
1,890,"[7025, 4240, 4181, 3220, 2981, 2257, 1088, 885...","[9253, 16498, 30276, 31964, 23283, 1575, 15809..."
2,943,"[42984, 39940, 39324, 38889, 38680, 36480, 358...","[9253, 16498, 15809, 31964, 30276, 6547, 1575,..."
3,1177,"[10087, 5114, 3784, 3702, 2904, 2449, 1698, 14...","[9253, 16498, 31964, 23283, 30276, 1575, 15809..."
4,1397,"[21557, 16528, 9000, 530, 270, 30, 33489, 2880...","[9253, 16498, 31964, 30276, 15809, 23283, 1575..."


In [ ]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["ideal", "recomms"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1 
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [ ]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.01407


Probamos entrenar el modelo durante 100 epocas: 

In [ ]:
%%time 
#Devuelve el tiempo que le lleva a mi maquina el entrenamiento
model = model.fit(interactions_matrix_csr, epochs = 100)

CPU times: total: 1min 33s
Wall time: 2min 33s


In [ ]:
from tqdm import tqdm

# Definimos dict donde vamos a ir almacenando las recomendaciones
recomms_dict = {
    'user_id': [],
    'recomms': [],
    'watched_content': []  # Nueva columna para contener el contenido que el usuario ya vio
}

# Obtenemos cantidad de usuarios y cantidad de items
n_users, n_items = interactions_matrix.shape
item_ids = np.arange(n_items)

# Por cada usuario del dataset de test, generamos recomendaciones
for user in tqdm(df_test.user_id.unique()):
    # Validar si el usuario se encuentra en la matriz de interacciones (interactions_matrix.index)
    if user in list(interactions_matrix.index):
        # Si el usuario está en train, no es cold start. Usamos el modelo para recomendar
        user_x = user_dict[user]  # Buscamos el índice del usuario en la matriz (transformamos id a índice)

        # Generar las predicciones para el usuario x
        preds = model.predict(user_ids=user_x, item_ids=item_ids)

        # Basándose en el ejemplo anterior, ordenar las predicciones de menor a mayor y quedarse con 50.
        scores = pd.Series(preds)
        scores.index = interactions_matrix.columns
        scores = list(pd.Series(scores.sort_values(ascending=False).index))[:50]

        # Obtener listado de contenidos vistos anteriormente por el usuario (en el set de train)
        watched_contents = df_train[df_train.user_id == user].anime_id.unique()

        # Filtrar contenidos ya vistos y quedarse con los primeros 10
        recomms = [x for x in scores if x not in watched_contents][:10]

        # Guardamos las recomendaciones en el diccionario
        recomms_dict['user_id'].append(user)
        recomms_dict['recomms'].append(scores)
        recomms_dict['watched_content'].append(watched_contents)

# Convertir el diccionario a un DataFrame
recomms_df = pd.DataFrame(recomms_dict)

100%|██████████| 1000/1000 [00:06<00:00, 151.16it/s]


In [ ]:
recomms_df = pd.DataFrame(recomms_dict)
recomms_df

,user_id,recomms,watched_content
0,36253,"[28223, 1535, 14813, 9989, 5081, 11757, 15809,...","[6324, 18195, 33502, 257, 2969, 18095, 9014, 3..."
1,128855,"[28223, 1535, 14813, 9989, 5081, 11757, 199, 2...","[3342, 32281, 2994, 21881, 16916, 1679, 6164, ..."
2,286479,"[28223, 1535, 14813, 9989, 5081, 11757, 20507,...","[45, 24531, 387, 10491, 31798, 25859, 23985, 4..."
3,221161,"[28223, 14813, 1535, 9989, 5081, 11757, 15809,...","[17251, 29501, 32238, 9332, 33, 21647, 31056, ..."
4,11697,"[28223, 1535, 14813, 9989, 5081, 11757, 199, 2...","[13855, 31715, 6984, 6736, 11021, 42990, 9253,..."
...,...,...,...
995,6149,"[28223, 14813, 1535, 9989, 5081, 11757, 199, 6...","[9289, 33741, 38080, 18247, 21067, 13333, 9936..."
996,35941,"[28223, 1535, 14813, 9989, 5081, 11757, 15809,...","[3058, 3382, 169, 14353, 5040, 963, 238, 31452..."
997,217920,"[28223, 1535, 14813, 9989, 5081, 11757, 20507,...","[101, 2899, 10681, 32698, 13839, 39988, 7222, ..."
998,135124,"[28223, 1535, 14813, 9989, 33206, 11757, 6547,...","[15633, 17873, 2889, 33219, 263, 37865, 15125,..."


In [ ]:
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="user_id", right_on="user_id")[["user_id", "anime_id", "recomms"]]
df_map.columns = ["user_id", "ideal", "recomms"]
df_map.head()

,user_id,ideal,recomms
0,781,"[36553, 35843, 34096, 31043, 21333, 16738, 153...","[28223, 1535, 14813, 9989, 5081, 11757, 199, 2..."
1,890,"[7025, 4240, 4181, 3220, 2981, 2257, 1088, 885...","[28223, 14813, 1535, 9989, 5081, 11757, 199, 1..."
2,943,"[42984, 39940, 39324, 38889, 38680, 36480, 358...","[1535, 28223, 14813, 9989, 5081, 20507, 11757,..."
3,1177,"[10087, 5114, 3784, 3702, 2904, 2449, 1698, 14...","[28223, 1535, 14813, 9989, 5081, 11757, 20507,..."
4,1397,"[21557, 16528, 9000, 530, 270, 30, 33489, 2880...","[28223, 1535, 14813, 9989, 5081, 11757, 20507,..."


In [ ]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["ideal", "recomms"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1 
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [ ]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.01482


En lugar de los 1.000 usuarios que mayor cantidad de animes vieron, probamos quedarnos con los primeros 10.000 usuarios para ver si mejoran las metricas

In [9]:
df_10000 = df_anime_count_sorted.head(10000)
df_10000

,user_id,amount_animes_watched
0,189037,15455
1,162615,14864
2,68042,13462
3,283786,12778
4,259790,9996
...,...,...
9995,297827,728
9996,22440,727
9997,229628,727
9998,276888,727


In [10]:
rating_complete.head()

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


Unimos ambos datasets:

In [11]:
# Realizamos la unión utilizando 'user_id' como clave y 'inner' como método
merged_df = pd.merge(rating_complete, df_10000, on='user_id', how='inner')

# Imprimimos el DataFrame resultante
print("Usuarios con 'user_id' en común:")
print(merged_df)

Usuarios con 'user_id' en común:
          user_id  anime_id  rating  amount_animes_watched
0             314     17397       7                    881
1             314     35849       8                    881
2             314      7338       8                    881
3             314      1575      10                    881
4             314     12685       5                    881
...           ...       ...     ...                    ...
11377072   353398     33421       6                    759
11377073   353398     37007       4                    759
11377074   353398     28825       6                    759
11377075   353398     34028       3                    759
11377076   353398     36296       8                    759

[11377077 rows x 4 columns]


In [ ]:
merged_df.shape

(11377077, 4)

In [ ]:
merged_df.head()

,user_id,anime_id,rating,amount_animes_watched
0,314,17397,7,881
1,314,35849,8,881
2,314,7338,8,881
3,314,1575,10,881
4,314,12685,5,881


Dividimos el dataset en conjuntos de train y test:

In [16]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test = train_test_split(merged_df, test_size=0.2, random_state=42)

In [ ]:
df_train.head()

,user_id,anime_id,rating,amount_animes_watched
1473788,44567,23023,4,1791
4523041,140663,29758,6,1493
3862552,120887,28025,2,1109
9725271,302508,1802,7,807
4010626,125583,13239,6,1187


In [ ]:
df_train.shape #Tenemos un conjunto de entrenamiento con 9.101.661 entradas

(9101661, 4)

In [ ]:
df_train.user_id.nunique() #En el set de train tenemos 10.000 usuarios diferentes

10000

In [ ]:
df_train.anime_id.nunique() #En el set de train tenemos 16.813 animes diferentes

16813

In [ ]:
df_test.head()

,user_id,anime_id,rating,amount_animes_watched
8623316,269503,22319,9,983
294120,9946,12671,1,1134
6316489,197099,16033,8,1594
11215942,348493,1727,4,868
9618668,299599,37999,7,896


In [ ]:
df_test.shape #Tenemos un conjunto de testeo con 2.275.416 entradas

(2275416, 4)

In [ ]:
df_test.user_id.nunique() #En el set de test tenemos 10.000 usuarios diferentes 

10000

Dado que en todo el set de datos tenemos 10.000 usuarios diferentes, entendemos que en este caso no habra usuarios COLD START, ya que todos los usuarios que se encuentran en train, se encuentran tambien en test

In [ ]:
#Comprobemos esto:
df_test[~df_test.user_id.isin(df_train.user_id.unique())].user_id.nunique()

0

Efectivamente, no tenemos usuarios que aparecen en el set de test pero no en el de train. Sino que todos los usuarios aparecen en ambos conjuntos.

In [ ]:
df_train.columns

Index(['user_id', 'anime_id', 'rating', 'amount_animes_watched'], dtype='object')

In [ ]:
#Nos quedamos con las columnas que nos interesan, tanto en train como en test: 
df_train = df_train.drop('amount_animes_watched', axis=1)
df_test = df_test.drop('amount_animes_watched', axis=1)

In [ ]:
df_train.head()

,user_id,anime_id,rating
1473788,44567,23023,4
4523041,140663,29758,6
3862552,120887,28025,2
9725271,302508,1802,7
4010626,125583,13239,6


In [ ]:
df_test.head()

,user_id,anime_id,rating
8623316,269503,22319,9
294120,9946,12671,1
6316489,197099,16033,8
11215942,348493,1727,4
9618668,299599,37999,7


Ahora, armamos la matriz de interacciones. Sabemos que tenemos 10.000 usuarios diferentes y 16.813 animes diferentes en el conjunto de train. 

In [24]:
matrix_columns = ['user_id', 'anime_id', 'rating']

In [ ]:
interactions = df_train[matrix_columns]
interactions.head()

,user_id,anime_id,rating
1473788,44567,23023,4
4523041,140663,29758,6
3862552,120887,28025,2
9725271,302508,1802,7
4010626,125583,13239,6


In [ ]:
interactions.shape

(9101661, 3)

In [ ]:
#Reseteamos los indices
interactions = interactions.reset_index(drop=True)

In [ ]:
interactions.head()

,user_id,anime_id,rating
0,44567,23023,4
1,140663,29758,6
2,120887,28025,2
3,302508,1802,7
4,125583,13239,6


In [ ]:
#Verificamos si existen filas duplicadas
duplicate_rows = interactions[interactions.duplicated(subset=['user_id', 'anime_id'], keep=False)]
print(duplicate_rows)

Empty DataFrame
Columns: [user_id, anime_id, rating]
Index: []


In [ ]:
interactions_matrix = interactions.pivot_table(index='user_id', columns='anime_id', values='rating')

In [ ]:
interactions_matrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,47307,47398,47402,47403,47410,47614,47616,47618,48177,48456
user_id,,,,,,,,,,,,,,,,,,,,,
314,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
326,10.0,10.0,8.0,8.0,NaN,8.0,8.0,NaN,9.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
371,9.0,9.0,7.0,6.0,NaN,7.0,NaN,5.0,7.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Completamos los NaN con 0s
interactions_matrix = interactions_matrix.fillna(0)
interactions_matrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,47307,47398,47402,47403,47410,47614,47616,47618,48177,48456
user_id,,,,,,,,,,,,,,,,,,,,,
314,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
326,10.0,10.0,8.0,8.0,0.0,8.0,8.0,0.0,9.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
346,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
371,9.0,9.0,7.0,6.0,0.0,7.0,0.0,5.0,7.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
interactions_matrix.shape #Tiene 10.000 filas (usuarios) y 16.813 columnas (animes)

(10000, 16813)

Vamos a transformar esta matriz sparsa a una matriz csr matrix (compressed sparse row)

In [32]:
from scipy.sparse import csr_matrix

In [ ]:
interactions_matrix_csr = csr_matrix(interactions_matrix.values)

In [ ]:
interactions_matrix_csr

<10000x16813 sparse matrix of type '<class 'numpy.float64'>'
	with 9101661 stored elements in Compressed Sparse Row format>

La siguiente celda generara un diccionario que nos servira mas adelante como indice. Tiene como keys el id de cada usuario y como valor el indice (numero de fila de la matriz)

In [34]:
user_ids = list(interactions_matrix.index)
user_dict = {}
counter = 0 
for i in user_ids:
    user_dict[i] = counter
    counter += 1

In [ ]:
user_dict

{314: 0,
 326: 1,
 340: 2,
 346: 3,
 371: 4,
 442: 5,
 446: 6,
 478: 7,
 493: 8,
 601: 9,
 603: 10,
 694: 11,
 730: 12,
 781: 13,
 785: 14,
 853: 15,
 876: 16,
 890: 17,
 911: 18,
 912: 19,
 936: 20,
 943: 21,
 953: 22,
 985: 23,
 995: 24,
 1008: 25,
 1177: 26,
 1187: 27,
 1231: 28,
 1289: 29,
 1393: 30,
 1397: 31,
 1407: 32,
 1453: 33,
 1469: 34,
 1470: 35,
 1483: 36,
 1612: 37,
 1627: 38,
 1629: 39,
 1694: 40,
 1697: 41,
 1731: 42,
 1766: 43,
 1803: 44,
 1817: 45,
 1878: 46,
 1938: 47,
 1946: 48,
 2003: 49,
 2018: 50,
 2021: 51,
 2037: 52,
 2162: 53,
 2235: 54,
 2236: 55,
 2292: 56,
 2303: 57,
 2414: 58,
 2464: 59,
 2483: 60,
 2721: 61,
 2885: 62,
 2892: 63,
 2938: 64,
 2976: 65,
 2993: 66,
 3139: 67,
 3151: 68,
 3165: 69,
 3198: 70,
 3303: 71,
 3333: 72,
 3334: 73,
 3436: 74,
 3473: 75,
 3502: 76,
 3533: 77,
 3578: 78,
 3608: 79,
 3644: 80,
 3681: 81,
 3701: 82,
 3784: 83,
 3812: 84,
 3838: 85,
 3940: 86,
 3999: 87,
 4016: 88,
 4059: 89,
 4104: 90,
 4134: 91,
 4152: 92,
 4312: 93,
 

Modelado: ya tenemos la matriz lista para entrenar el modelo. Utilizaremos la libreria LFM (light fm)

In [35]:
from lightfm import LightFM

c:\Users\ferbi\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [ ]:
model = LightFM(no_components=3, random_state=100, learning_rate=0.01)

Entrenamos este modelo durante 10 epocas:

In [ ]:
%%time 
#Devuelve el tiempo que le lleva a mi maquina el entrenamiento
model = model.fit(interactions_matrix_csr, epochs = 10)

CPU times: total: 1min 6s
Wall time: 2min 19s


Obtenemos las recomendaciones para cada usuario en el set de test, en este caso, al no tener usuarios cold start, tampoco haremos recomendaciones del contenido mas popular:

In [ ]:
from tqdm import tqdm

# Definimos dict donde vamos a ir almacenando las recomendaciones
recomms_dict = {
    'user_id': [],
    'recomms': [],
    'watched_content': []  # Nueva columna para contener el contenido que el usuario ya vio
}

# Obtenemos cantidad de usuarios y cantidad de items
n_users, n_items = interactions_matrix.shape
item_ids = np.arange(n_items)

# Por cada usuario del dataset de test, generamos recomendaciones
for user in tqdm(df_test.user_id.unique()):
    # Validar si el usuario se encuentra en la matriz de interacciones (interactions_matrix.index)
    if user in list(interactions_matrix.index):
        # Si el usuario está en train, no es cold start. Usamos el modelo para recomendar
        user_x = user_dict[user]  # Buscamos el índice del usuario en la matriz (transformamos id a índice)

        # Generar las predicciones para el usuario x
        preds = model.predict(user_ids=user_x, item_ids=item_ids)

        # Basándose en el ejemplo anterior, ordenar las predicciones de menor a mayor y quedarse con 50.
        scores = pd.Series(preds)
        scores.index = interactions_matrix.columns
        scores = list(pd.Series(scores.sort_values(ascending=False).index))[:50]

        # Obtener listado de contenidos vistos anteriormente por el usuario (en el set de train)
        watched_contents = df_train[df_train.user_id == user].anime_id.unique()

        # Filtrar contenidos ya vistos y quedarse con los primeros 10
        recomms = [x for x in scores if x not in watched_contents][:10]

        # Guardamos las recomendaciones en el diccionario
        recomms_dict['user_id'].append(user)
        recomms_dict['recomms'].append(scores)
        recomms_dict['watched_content'].append(watched_contents)

# Convertir el diccionario a un DataFrame
recomms_df = pd.DataFrame(recomms_dict)


100%|██████████| 10000/10000 [02:52<00:00, 57.93it/s]


In [ ]:
recomms_df = pd.DataFrame(recomms_dict)
recomms_df

,user_id,recomms,watched_content
0,269503,"[1535, 11757, 6547, 9989, 16498, 4224, 9253, 5...","[36043, 19363, 36949, 16662, 2950, 1861, 9062,..."
1,9946,"[1535, 11757, 6547, 9253, 4224, 1575, 9989, 50...","[21899, 35848, 23423, 3603, 7817, 1887, 12049,..."
2,197099,"[1535, 11757, 6547, 9989, 5081, 9253, 1575, 42...","[21273, 36919, 39819, 36511, 21879, 36497, 958..."
3,348493,"[1535, 11757, 6547, 16498, 9989, 5081, 4224, 9...","[1920, 9332, 7059, 358, 6127, 4884, 1013, 376,..."
4,299599,"[11757, 6547, 1535, 30276, 1575, 4224, 16498, ...","[6291, 32937, 9203, 31964, 31163, 21327, 4535,..."
...,...,...,...
9995,253316,"[1535, 11757, 6547, 16498, 9989, 5081, 4224, 9...","[2798, 2132, 10163, 2768, 8129, 940, 39587, 32..."
9996,30664,"[1535, 11757, 6547, 16498, 9989, 5081, 4224, 9...","[19815, 245, 3901, 26023, 23945, 585, 20365, 1..."
9997,258866,"[1535, 11757, 6547, 9989, 9253, 4224, 1575, 16...","[30370, 31772, 40591, 17849, 38810, 20507, 415..."
9998,13676,"[11757, 6547, 1535, 30276, 1575, 4224, 16498, ...","[14749, 24873, 5041, 32935, 22877, 17505, 4789..."


EVALUACION DE METRICAS  
Compararemos nuestras recomendaciones contra las que los usuarios realemnte vieron (set de test)

Primero ordenamos los animes segun los ratings (de mayor a menor) que dieron los usuarios en el set de testeo. Luego, agrupamos y arnamamos un listado de animes para cada usuario. Este sera el listado "ideal" contra el que vams a comparar nuestras recomendaciones.

In [ ]:
ideal_recomms = df_test\
                  .sort_values(by=["user_id", "rating", "anime_id"], ascending=False)\
                  .groupby(["user_id"], as_index=False)\
                  .agg({"anime_id": lambda x: list(x.unique())})

Generamos el dataset ideal

In [ ]:
#Tenemos un conjunto de datos que muestra las recomendaciones
#ideales para cada uno de los usuarios del conjunto de testeo
ideal_recomms.head()

,user_id,anime_id
0,314,"[21001, 14199, 13851, 13663, 13221, 12549, 113..."
1,326,"[37675, 35073, 34599, 32827, 31240, 5300, 5258..."
2,340,"[10568, 9253, 40221, 37105, 36949, 35507, 3413..."
3,346,"[37510, 33253, 31580, 31376, 28621, 23281, 222..."
4,371,"[164, 40776, 1430, 1088, 572, 32, 38993, 35968..."


Vamos a medir MAP: 
Primero unimos en un mismo dataframe las recomndaciones con el set ideal 

In [ ]:
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="user_id", right_on="user_id")[["user_id", "anime_id", "recomms"]]
df_map.columns = ["user_id", "ideal", "recomms"]
df_map.head()

,user_id,ideal,recomms
0,314,"[21001, 14199, 13851, 13663, 13221, 12549, 113...","[1535, 11757, 6547, 16498, 9989, 5081, 4224, 9..."
1,326,"[37675, 35073, 34599, 32827, 31240, 5300, 5258...","[1535, 11757, 6547, 16498, 9989, 4224, 5081, 9..."
2,340,"[10568, 9253, 40221, 37105, 36949, 35507, 3413...","[1535, 11757, 6547, 16498, 9989, 5081, 4224, 9..."
3,346,"[37510, 33253, 31580, 31376, 28621, 23281, 222...","[11757, 1535, 6547, 4224, 9253, 1575, 9989, 50..."
4,371,"[164, 40776, 1430, 1088, 572, 32, 38993, 35968...","[1535, 11757, 6547, 16498, 9989, 4224, 5081, 9..."


In [ ]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["ideal", "recomms"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1 
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [ ]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.02044


Las métricas mejoran pero no tanto.  

Probemos entrenando un modelo con los 20.000 usuarios que mas animes vieron:

In [12]:
df_20000 = df_anime_count_sorted.head(20000)
df_20000

,user_id,amount_animes_watched
0,189037,15455
1,162615,14864
2,68042,13462
3,283786,12778
4,259790,9996
...,...,...
19995,43481,535
19996,192136,535
19997,292161,535
19998,202209,535


Unimos ambos datasets:

In [13]:
# Realizamos la unión utilizando 'user_id' como clave y 'inner' como método
merged_df = pd.merge(rating_complete, df_20000, on='user_id', how='inner')

# Imprimimos el DataFrame resultante
print("Usuarios con 'user_id' en común:")
print(merged_df)

Usuarios con 'user_id' en común:
          user_id  anime_id  rating  amount_animes_watched
0              17      5781       7                    709
1              17      6682      10                    709
2              17     14199       2                    709
3              17     12355       6                    709
4              17     23673       6                    709
...           ...       ...     ...                    ...
17546513   353398     33421       6                    759
17546514   353398     37007       4                    759
17546515   353398     28825       6                    759
17546516   353398     34028       3                    759
17546517   353398     36296       8                    759

[17546518 rows x 4 columns]


In [14]:
merged_df.head()

,user_id,anime_id,rating,amount_animes_watched
0,17,5781,7,709
1,17,6682,10,709
2,17,14199,2,709
3,17,12355,6,709
4,17,23673,6,709


Dividimos el dataset en conjuntos de train y test:

In [17]:
df_train, df_test = train_test_split(merged_df, test_size=0.2, random_state=42)

In [18]:
df_train.user_id.nunique() #En el set de train tenemos 20.000 usuarios diferentes

20000

In [19]:
df_train.anime_id.nunique() #En el set de train tenemos 16.819 animes diferentes

16819

In [20]:
df_test.user_id.nunique() #En el set de test tenemos 20.000 usuarios diferentes 

20000

Dado que en todo el set de datos tenemos 20.000 usuarios diferentes, entendemos que en este caso no habra usuarios COLD START, ya que todos los usuarios que se encuentran en train, se encuentran tambien en test

In [21]:
#Comprobemos esto:
df_test[~df_test.user_id.isin(df_train.user_id.unique())].user_id.nunique()

0

Efectivamente, no tenemos usuarios que aparecen en el set de test pero no en el de train. Sino que todos los usuarios aparecen en ambos conjuntos.

In [22]:
#Nos quedamos con las columnas que nos interesan, tanto en train como en test: 
df_train = df_train.drop('amount_animes_watched', axis=1)
df_test = df_test.drop('amount_animes_watched', axis=1)

Ahora, armamos la matriz de interacciones. Sabemos que tenemos 20.000 usuarios diferentes y 16.819 animes diferentes en el conjunto de train. 

In [25]:
interactions = df_train[matrix_columns]
interactions.head()

,user_id,anime_id,rating
9698839,196485,36475,7
12120288,245893,317,8
5727071,115661,8246,5
10427489,211237,9521,7
5020636,101155,1047,10


In [26]:
#Reseteamos los indices
interactions = interactions.reset_index(drop=True)

In [27]:
interactions_matrix = interactions.pivot_table(index='user_id', columns='anime_id', values='rating')

In [28]:
interactions_matrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,47307,47398,47402,47403,47410,47614,47616,47618,48177,48456
user_id,,,,,,,,,,,,,,,,,,,,,
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,8.0,6.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#Completamos los NaN con 0s
interactions_matrix = interactions_matrix.fillna(0)
interactions_matrix.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,47307,47398,47402,47403,47410,47614,47616,47618,48177,48456
user_id,,,,,,,,,,,,,,,,,,,,,
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,8.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
interactions_matrix.shape #Tiene 20.000 filas (usuarios) y 16.819 columnas (animes)

(20000, 16819)

In [33]:
interactions_matrix_csr = csr_matrix(interactions_matrix.values)

Modelado: ya tenemos la matriz lista para entrenar el modelo. Utilizaremos la libreria LFM (light fm)

In [36]:
model = LightFM(no_components=3, random_state=100, learning_rate=0.01)

In [37]:
%%time 
#Devuelve el tiempo que le lleva a mi maquina el entrenamiento
model = model.fit(interactions_matrix_csr, epochs = 10)

CPU times: total: 1min 24s
Wall time: 3min 41s


Obtenemos las recomendaciones para cada usuario en el set de test, en este caso, al no tener usuarios cold start, tampoco haremos recomendaciones del contenido mas popular:

In [38]:
from tqdm import tqdm

# Definimos dict donde vamos a ir almacenando las recomendaciones
recomms_dict = {
    'user_id': [],
    'recomms': [],
    'watched_content': []  # Nueva columna para contener el contenido que el usuario ya vio
}

# Obtenemos cantidad de usuarios y cantidad de items
n_users, n_items = interactions_matrix.shape
item_ids = np.arange(n_items)

# Por cada usuario del dataset de test, generamos recomendaciones
for user in tqdm(df_test.user_id.unique()):
    # Validar si el usuario se encuentra en la matriz de interacciones (interactions_matrix.index)
    if user in list(interactions_matrix.index):
        # Si el usuario está en train, no es cold start. Usamos el modelo para recomendar
        user_x = user_dict[user]  # Buscamos el índice del usuario en la matriz (transformamos id a índice)

        # Generar las predicciones para el usuario x
        preds = model.predict(user_ids=user_x, item_ids=item_ids)

        # Basándose en el ejemplo anterior, ordenar las predicciones de menor a mayor y quedarse con 50.
        scores = pd.Series(preds)
        scores.index = interactions_matrix.columns
        scores = list(pd.Series(scores.sort_values(ascending=False).index))[:50]

        # Obtener listado de contenidos vistos anteriormente por el usuario (en el set de train)
        watched_contents = df_train[df_train.user_id == user].anime_id.unique()

        # Filtrar contenidos ya vistos y quedarse con los primeros 10
        recomms = [x for x in scores if x not in watched_contents][:10]

        # Guardamos las recomendaciones en el diccionario
        recomms_dict['user_id'].append(user)
        recomms_dict['recomms'].append(scores)
        recomms_dict['watched_content'].append(watched_contents)

# Convertir el diccionario a un DataFrame
recomms_df = pd.DataFrame(recomms_dict)

100%|██████████| 20000/20000 [11:25<00:00, 29.18it/s]


In [39]:
recomms_df = pd.DataFrame(recomms_dict)
recomms_df

,user_id,recomms,watched_content
0,26661,"[6547, 11757, 16498, 15809, 1575, 1535, 30276,...","[705, 435, 6045, 15377, 17157, 9723, 1568, 577..."
1,34764,"[11757, 6547, 1575, 16498, 4224, 15809, 20507,...","[5545, 40147, 2505, 3452, 23137, 41824, 4720, ..."
2,5809,"[11757, 6547, 1575, 4224, 16498, 30276, 1535, ...","[34626, 27633, 36896, 37520, 31741, 37345, 241..."
3,182279,"[16498, 6547, 11757, 30276, 4224, 15809, 1575,...","[30544, 35756, 38101, 25099, 30383, 35968, 379..."
4,343093,"[11757, 6547, 1575, 4224, 16498, 1535, 9253, 3...","[5355, 13759, 249, 4106, 19221, 1536, 1594, 76..."
...,...,...,...
19995,117164,"[11757, 6547, 1575, 4224, 16498, 1535, 30276, ...","[36108, 9260, 2787, 10080, 32268, 22687, 38993..."
19996,343145,"[11757, 6547, 16498, 1575, 4224, 1535, 30276, ...","[31645, 603, 25781, 8937, 740, 16273, 2126, 11..."
19997,264944,"[11757, 6547, 16498, 1575, 4224, 30276, 15809,...","[18495, 10711, 24991, 355, 10020, 64, 65, 3604..."
19998,295882,"[11757, 6547, 1575, 16498, 4224, 30276, 1535, ...","[22273, 40046, 3652, 37095, 37451, 32915, 1072..."


Primero ordenamos los animes segun los ratings (de mayor a menor) que dieron los usuarios en el set de testeo. Luego, agrupamos y arnamamos un listado de animes para cada usuario. Este sera el listado "ideal" contra el que vams a comparar nuestras recomendaciones.

In [40]:
ideal_recomms = df_test\
                  .sort_values(by=["user_id", "rating", "anime_id"], ascending=False)\
                  .groupby(["user_id"], as_index=False)\
                  .agg({"anime_id": lambda x: list(x.unique())})

In [41]:
#Tenemos un conjunto de datos que muestra las recomendaciones
#ideales para cada uno de los usuarios del conjunto de testeo
ideal_recomms.head()

,user_id,anime_id
0,17,"[28677, 23755, 16498, 16035, 15451, 14467, 125..."
1,19,"[30276, 28851, 18523, 14227, 12113, 11741, 116..."
2,42,"[31043, 22319, 22199, 16782, 14713, 8676, 6746..."
3,111,"[31772, 28999, 28851, 28171, 11577, 10218, 100..."
4,146,"[38555, 38524, 37379, 36949, 36220, 34530, 282..."


Vamos a medir MAP: 
Primero unimos en un mismo dataframe las recomndaciones con el set ideal 

In [42]:
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="user_id", right_on="user_id")[["user_id", "anime_id", "recomms"]]
df_map.columns = ["user_id", "ideal", "recomms"]
df_map.head()

,user_id,ideal,recomms
0,17,"[28677, 23755, 16498, 16035, 15451, 14467, 125...","[6547, 11757, 1535, 16498, 1575, 4224, 5081, 1..."
1,19,"[30276, 28851, 18523, 14227, 12113, 11741, 116...","[11757, 6547, 16498, 1575, 4224, 30276, 19815,..."
2,42,"[31043, 22319, 22199, 16782, 14713, 8676, 6746...","[11757, 6547, 1575, 16498, 4224, 30276, 15809,..."
3,111,"[31772, 28999, 28851, 28171, 11577, 10218, 100...","[11757, 6547, 1575, 16498, 4224, 1535, 30276, ..."
4,146,"[38555, 38524, 37379, 36949, 36220, 34530, 282...","[11757, 6547, 16498, 1575, 4224, 30276, 19815,..."


In [43]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["ideal", "recomms"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1 
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [44]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.02245
